In [1]:
import pandas as pd
from datetime import datetime
from utils.scalability_utils import cluster_to_dataframe
from models.JGSMF_with_phases import solve_with_phases
from models.SSPMF import SSPMFModel 

In [2]:
cluster_directory = "../data/MTSP/Laporte/Tabela4/"

df = cluster_to_dataframe(cluster_directory)

In [3]:
print(df['num_tools'].unique())
print(df['magazine_capacity'].unique())


[15 20 25]
[ 5 15 10 20]


In questo dataset tutti le istanze hanno 9 job e un numero di tools tra 15, 20 o 25. La capienza invece è tra 5, 10, 15 o 20.
Per l'analisi di scalabilità ho deciso di prendere tutte le combinazioni di numero di tools e capienza, per poi valutarne la velocità di esecuzione tra i 2 modelli

In [4]:
scalability_df = df.drop_duplicates(subset=['num_tools', 'magazine_capacity'])

scalability_df

,filename,num_jobs,num_tools,magazine_capacity,job_tools
0,L1-1.txt,9,15,5,"{1: [10, 12], 2: [3, 8, 12, 13, 14], 3: [4, 10..."
10,L10-1.txt,9,20,15,"{1: [1, 3, 6, 11, 13, 14, 16, 18, 19, 20], 2: ..."
30,L12-1.txt,9,25,10,"{1: [1, 4, 9, 10, 18, 20, 25], 2: [3, 5, 6, 7,..."
50,L14-1.txt,9,25,15,"{1: [2, 4, 7, 12, 15, 16, 17, 21, 23, 25], 2: ..."
90,L18-1.txt,9,25,20,"{1: [1, 4, 9, 10, 13, 16, 18, 19, 20, 22], 2: ..."
110,L2-1.txt,9,15,10,"{1: [2, 8, 9, 11, 13], 2: [8, 10], 3: [3, 7], ..."
190,L27-1.txt,9,20,5,"{1: [6, 11], 2: [1, 2, 5, 16, 20], 3: [3, 4, 7..."
200,L28-1.txt,9,20,10,"{1: [14, 16, 18, 19], 2: [2, 5, 13, 15, 19], 3..."
230,L30-1.txt,9,25,5,"{1: [11, 23, 25], 2: [9, 23], 3: [2, 3, 10, 21..."


Ogni esperimento su queste istanze verrà ripetuto 5 volte per ottenenere la mediana del tempo di ogni combinazione. Poi verrà costruito il grafico della scalability con questi dati

In [ ]:
results = []

#SSPMF Model
print("SSPMF Model")
for index,row in scalability_df.iterrows():

    print("Instance: ", row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
        SSPMF_model.optimize()
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    results.append(["SSPMF",num_jobs, num_tools, magazine_capacity, median_time])

print("-----------------------------------")
#JGSMF Model
print("JGSMF Model")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    median_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        job_order, switches = solve_with_phases(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600)
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    median_time = pd.Series(same_iteration_times).median()
    results.append(["JGSMF",num_jobs, num_tools, magazine_capacity, median_time])

results_df = pd.DataFrame(results, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])







Working on  L1-1.txt
Iteration  0
Optimal solution found.
Elapsed time:  13.651906
Iteration  1
Optimal solution found.
Elapsed time:  13.599681
Iteration  2
Optimal solution found.
Elapsed time:  12.706919
Iteration  3
Optimal solution found.
Elapsed time:  12.860311
Iteration  4
Optimal solution found.
Elapsed time:  13.27253
Working on  L10-1.txt
Iteration  0
Optimal solution found.
Elapsed time:  34.443779
Iteration  1
Optimal solution found.
Elapsed time:  31.335046
Iteration  2
Optimal solution found.
Elapsed time:  31.35798
Iteration  3
Optimal solution found.
Elapsed time:  31.451951
Iteration  4
Optimal solution found.
Elapsed time:  32.890856
Working on  L12-1.txt
Iteration  0
Optimal solution found.
Elapsed time:  39.384636
Iteration  1
Optimal solution found.
Elapsed time:  39.721033
Iteration  2
Optimal solution found.
Elapsed time:  35.629512
Iteration  3
Optimal solution found.
Elapsed time:  35.327065
Iteration  4
Optimal solution found.
Elapsed time:  38.689601
Working

Dato che alcuni esperimenti sembrano più veloci di quanto dovrebbero (considerando le combinazioni di parametri), la cui rapidità è probabilmente dovuto ai tools richiesti da ogni job .Ora ripeterò l'esperimento eseguito sopra solo considerando 5 diverse istanze per combinazione ed eseguendone poi la mediana sulla mediana dei tempi, in modo da cercare di mitigare questo problema

In [5]:
new_scalability_df = df.groupby(['num_tools', 'magazine_capacity']).head(5)


new_scalability_df

,filename,num_jobs,num_tools,magazine_capacity,job_tools
0,L1-1.txt,9,15,5,"{1: [10, 12], 2: [3, 8, 12, 13, 14], 3: [4, 10..."
1,L1-10.txt,9,15,5,"{1: [4, 5, 10, 13], 2: [5, 9, 13], 3: [2, 4, 1..."
2,L1-2.txt,9,15,5,"{1: [3, 6, 10, 11, 12], 2: [1, 4, 8, 13], 3: [..."
3,L1-3.txt,9,15,5,"{1: [3, 10, 12, 13, 15], 2: [4, 10, 12, 15], 3..."
4,L1-4.txt,9,15,5,"{1: [2, 4, 10, 12], 2: [5, 9, 10, 11, 12], 3: ..."
10,L10-1.txt,9,20,15,"{1: [1, 3, 6, 11, 13, 14, 16, 18, 19, 20], 2: ..."
11,L10-10.txt,9,20,15,"{1: [1, 3, 8, 10, 12, 16, 19], 2: [5, 7, 12, 1..."
12,L10-2.txt,9,20,15,"{1: [1, 2, 3, 5, 6, 8, 10, 12, 14, 15, 16, 17,..."
13,L10-3.txt,9,20,15,"{1: [5, 11, 12, 16, 17, 19], 2: [7, 8, 10, 13,..."
14,L10-4.txt,9,20,15,"{1: [3, 4, 9, 10, 12, 14, 15, 17, 18], 2: [1, ..."


In [ ]:
new_results_df = []
# SSPMF Model
print("SSPMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
            SSPMF_model.optimize()
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        median_time = pd.Series(iteration_times).median()
        same_iteration_times.append(median_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["SSPMF", num_jobs, num_tools, magazine_capacity, overall_median_time])

print("-----------------------------------")
# JGSMF Model
print("JGSMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            job_order, switches = solve_with_phases(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600)
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        median_time = pd.Series(iteration_times).median()
        same_iteration_times.append(median_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["JGSMF", num_jobs, num_tools, magazine_capacity, overall_median_time])

new_results_df = pd.DataFrame(new_results_df, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])

Group  15 5
Instance L1-1.txt
Optimal solution found.
Instance L1-10.txt
Optimal solution found.
Instance L1-2.txt
Optimal solution found.
Instance L1-3.txt
Optimal solution found.
Instance L1-4.txt
Optimal solution found.
Group  15 10
Instance L2-1.txt
Optimal solution found.
Instance L2-10.txt
Optimal solution found.
Instance L2-2.txt
Optimal solution found.
Instance L2-3.txt
Optimal solution found.
Instance L2-4.txt
Optimal solution found.
Group  20 5
Instance L27-1.txt
Optimal solution found.
Instance L27-10.txt
Optimal solution found.
Instance L27-2.txt
Optimal solution found.
Instance L27-3.txt
Optimal solution found.
Instance L27-4.txt
Optimal solution found.
Group  20 10
Instance L28-1.txt
Optimal solution found.
Instance L28-10.txt
Optimal solution found.
Instance L28-2.txt
Optimal solution found.
Instance L28-3.txt
Optimal solution found.
Instance L28-4.txt
Optimal solution found.
Group  20 15
Instance L10-1.txt
Optimal solution found.
Instance L10-10.txt
Optimal solution fou